In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

class style :
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# check python bit version
import platform
print(platform.architecture())

!pip install jupyternotify
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

!pip install email-notify-magic
%load_ext email_notify_magic

('64bit', 'WindowsPE')


<IPython.core.display.Javascript object>

In [2]:
%%notify
%%time

Review= pd.read_excel(r"C:\\Users\\rishabh.pandey\\Downloads\\Tasks\\Task11_Products(5Apr)\Products.xlsx", sheet_name=3, header=0)
Review= Review[['ProductID', 'ReviewText', 'ReviewStar']]
# Review['ReviewText']=Review['ReviewText'].apply(str).apply(lambda x: clean(x))
# Review.to_csv(r"C:\\Users\\rishabh.pandey\\Downloads\\Tasks\\Task11_Products(5Apr)\Review.csv")
print(Review.shape)

(50887, 3)
Wall time: 33.8 s


<IPython.core.display.Javascript object>

## Cleaning: Code

In [ ]:
%%notify
%%time

from nltk.corpus import wordnet
from nltk import word_tokenize

def clean(text):
    
    text_token = word_tokenize(text)
    text_token2 = [x.lower() for x in text_token]
    
    # Mapping Social-media shortcuts
    shortcuts = {'u': 'you','lol': 'laughing out loud', 'k': 'okay', 'm': 'am', 'b4': 'before',
                'wlcm': 'welcome', 'bc': 'because', '<3': 'love', 'xoxo': 'love','gr8': 'great', 'bday': 'birthday',
                 'awsm': 'awesome', 'gud': 'good', 'h8': 'hate','irl': 'in real life', 'yolo': 'you only live once','btw': 'by the way'}
    nsw=pd.Series(text_token)
    words=nsw.map(shortcuts).fillna(nsw)
    
    #Get an integrated list of listed_words (words like yolo must be combined, You Only Live Once :)
    words= [word.split(' ') for word in words]    
    import itertools
    words= list(itertools.chain.from_iterable(words))
    
    # lemmatize the words with appropriate POS Tag
    def pos_tag(word):  
        tag= nltk.pos_tag([word])[0][1][0].upper()
        dic= {"J": wordnet.ADJ,"N": wordnet.NOUN,"V": wordnet.VERB,"R": wordnet.ADV}
        return dic.get(tag, wordnet.NOUN)
    words = [WordNetLemmatizer().lemmatize(word, pos=pos_tag(word)) for word in words] 
    words2=[word for word in words if nltk.pos_tag([word])[0][1] not in ['VB','VBG','VBD','VBN','VBP','VBZ','CC','MD','IN']]
    
    '''# perform stemming for non lemmatized words (like 'cleaning')
    from nltk.stem.snowball import SnowballStemmer
    
    englishStemmer=SnowballStemmer("english")
    words=[englishStemmer.stem(word) for  word in words]'''
    
    '''words=[word.encode('utf-8') for word in words]
    words=[hnspl.suggest(word)[0] if hnspl.spell(word)==False else word for word in words]'''
    
    clean_text=" ".join(words2)
    return clean_text

In [3]:
# !pip install aspect-based-sentiment-analysis --user
import aspect_based_sentiment_analysis as absa
nlp= absa.load()

## Test

In [6]:
text = ("We are great fans of Slack, but we wish the subscriptions were more accessible to small startups.")

slack, price = nlp(text, aspects=['slack', 'price'])
print(price.sentiment == absa.Sentiment.negative)
print(slack.sentiment == absa.Sentiment.positive)

True
True


In [8]:
text = "I love nokia\'s battery but also it's camera"

battery, camera = nlp(text, aspects=['battery', 'camera'])
print(battery.sentiment == absa.Sentiment.positive)
print(camera.sentiment == absa.Sentiment.positive)

True
True


In [10]:
'''def om(text):
    text_token = word_tokenize(text)
    pos=nltk.pos_tag(text_token)
    words=[i[0] for i in pos if i[1] in ['NN','NNP','NNS','NNPS']]
    return " ".join(words)'''

'def om(text):\n    text_token = word_tokenize(text)\n    pos=nltk.pos_tag(text_token)\n    words=[i[0] for i in pos if i[1] in [\'NN\',\'NNP\',\'NNS\',\'NNPS\']]\n    return " ".join(words)'